# Basit Doğrusal Regresyon - Model Tuning - Model Doğrulama

### Bir internet hizmet sağlayıcısı, kullanıcılarının hız testlerini kaydetmek istiyor. Şirket, kullanıcının internet bağlantısının hızını etkileyen faktörleri anlamak için bir veri seti toplamaya karar verdi. Toplanan veri seti, kullanıcının internet bağlantısı hızı ve aşağıdaki faktörler hakkında bilgi içermektedir.

### İnternet paket tipi (ADSL, VDSL, fiber, kablo vb.)
### Modem Markası
### Modem Modeli
### Modem Yaşı
### Bağlantı Tipi (kablolu veya kablosuz)
### İnternet Kullanım Saati
### İnternet Kullanımı Sırasındaki Cihaz Sayısı

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
# veri setinin oluşturulması.

np.random.seed(42)
n = 500  # veri setindeki gözlem sayısı

# Bağımsız değişkenlerin olası değerleri
internet_paketleri = ['ADSL', 'VDSL', 'fiber', 'kablo']
modem_markalari = ['TP-Link', 'Netgear', 'Linksys', 'D-Link']
modem_modelleri = ['WRT54GL', 'Archer C9', 'DIR-655', 'Nighthawk X10']
baglanti_tipleri = ['kablolu', 'kablosuz']

# Bağımsız değişkenleri içeren rastgele veri seti oluşturun
veri = pd.DataFrame({
    'internet_paketi': np.random.choice(internet_paketleri, size=n),
    'modem_markasi': np.random.choice(modem_markalari, size=n),
    'modem_modeli': np.random.choice(modem_modelleri, size=n),
    'modem_yasi': np.random.randint(low=1, high=5, size=n),
    'baglanti_tipi': np.random.choice(baglanti_tipleri, size=n),
    'internet_kullanim_saati': np.random.randint(low=1, high=24, size=n),
    'cihaz_sayisi': np.random.randint(low=1, high=6, size=n),
})

# Hedef değişken (internet hızı) oluşturun
veri['internet_hizi'] = 10 + \
    veri.modem_yasi * 2 + \
    (veri.internet_kullanim_saati / 24) * 10 + \
    (veri.cihaz_sayisi - 1) * (-2) + \
    np.random.normal(scale=5, size=n)

# Veri setini görüntüleyin
veri.head()

,internet_paketi,modem_markasi,modem_modeli,modem_yasi,baglanti_tipi,internet_kullanim_saati,cihaz_sayisi,internet_hizi
0,fiber,Linksys,Archer C9,3,kablolu,14,5,14.332861
1,kablo,TP-Link,DIR-655,2,kablosuz,15,2,18.979808
2,ADSL,TP-Link,WRT54GL,2,kablosuz,14,3,13.511732
3,fiber,D-Link,WRT54GL,3,kablosuz,7,2,15.186245
4,fiber,Linksys,WRT54GL,3,kablosuz,17,5,18.725683


#### Elde ettiğimiz veri setinde, hedef değişken 'internet_hizi' dir. Bu değişkeni diğer bağımsız değişkenlerle ilişkilendirmek için basit doğrusal regresyon kullanabiliriz.

#### Basit doğrusal regresyon, yalnızca bir bağımsız değişken ile hedef değişken arasındaki ilişkiyi modellemektedir. Bu nedenle, en basit modelimizi 'modem_yasi' bağımsız değişkeni ile 'internet_hizi' hedef değişkeni arasındaki ilişkiyi inceleyerek oluşturabiliriz.

In [17]:
# bağımlı ve bağımsız değişkenlerin oluşturulması

X = veri['modem_yasi'].values.reshape(-1, 1)
y = veri['internet_hizi'].values.reshape(-1, 1)

In [18]:
# eğitim ve test setlerinin ayrıştırılması.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# modelimizi oluşturalım ve eğitelim.

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [20]:
# Test verileri üzerinde tahmin yapalım.

y_pred = model.predict(X_test)

In [21]:
# Modelin performansını değerlendirelim.

mse = mean_squared_error(y_test, y_pred)
r_kare = r2_score(y_test, y_pred)

print('Ortalama Hata Kareleri (MSE): ', mse)
print('R-kare değeri: ', r_kare)

Ortalama Hata Kareleri (MSE):  42.88050268926186
R-kare değeri:  -0.15242221712080806


#### Model tuning işlemi için, modelimizdeki tek hiperparametre olan rastgele durum (random_state) parametresini değiştirerek farklı sonuçlar elde edebiliriz.

In [29]:
# Model tuning işlemi.

model1 = LinearRegression().fit(X_train, y_train)
model2 = LinearRegression().fit(X_train, y_train)
model3 = LinearRegression().fit(X_train, y_train)

modeller = [model1, model2, model3]
random_states = [42, 0, 100]

for i in range(3):
    model = modeller[i]
    random_state = random_states[i]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    print(f'Model {i+1}, rastgele durum {random_state} için: ')
    print('MSE: ', mse)
    
    print('')

Model 1, rastgele durum 42 için: 
MSE:  42.88050268926186

Model 2, rastgele durum 0 için: 
MSE:  42.88050268926186

Model 3, rastgele durum 100 için: 
MSE:  42.88050268926186



#### Sonuç olarak, farklı rastgele durumlar kullanılarak oluşturulan modellerin performanslarını değerlendirdik. Her modelin performansı farklı olabilir. Bu nedenle farklı rastgele durumlar kullanarak model tuning yapmak genellikle iyi bir fikirdir.

#### Ayrıca, model doğrulama işlemini gerçekleştirmek için çeşitli teknikler kullanılabilir. Örneğin, cross_validation gibi tekniklerle modelimizin performansını daha iyi değerlendirebiliriz. Ancak, bu teknikleri kullanmak, modelimizi daha karmaşık hale getirecektir. Bu nedenle, basit doğrusal regresyon modelimiz için model tuning ve basit bir model doğrulama işlemi yeterlidir.